In [2]:
import rasterio
import numpy as np
import pandas as pd

dem_files = [
    "P5_PAN_Maharashtra_N16_000_E074_000_DEM_30m.tif",
    "P5_PAN_AndhraPradesh_N18_000_E083_000_DEM_30m.tif",
    "P5_PAN_Odisha_N21_000_E083_000_DEM_30m.tif",
    "P5_PAN_Odisha_N21_000_E084_000_DEM_30m.tif",
    "P5_PAN_Chattisgarh_N22_000_E082_000_DEM_30m.tif",
    "P5_PAN_Jharkhand_N23_000_E086_000_DEM_30m.tif",
    "P5_PAN_WestBengal_N23_000_E087_000_DEM_30m.tif",
    "P5_PAN_MadhyaPradesh_N24_000_E082_000_DEM_30m.tif"
]

all_data = []
step = 10  # Sample only every 10th pixel in both directions

for dem_path in dem_files:
    print(f"Processing {dem_path}...")
    with rasterio.open(dem_path) as src:
        dem = src.read(1)
        transform = src.transform
        nodata = src.nodata
        rows, cols = dem.shape
        # Calculate slope and aspect
        x, y = np.gradient(dem, src.res[0], src.res[1])
        slope = np.degrees(np.arctan(np.sqrt(x**2 + y**2)))
        aspect = np.degrees(np.arctan2(-x, y))
        aspect = np.where(aspect < 0, 360 + aspect, aspect)
        loc_label = dem_path.split('_')[2]  # e.g. N16, N18, etc.

        # Only loop over every nth row/col for speed
        for r in range(0, rows, step):
            for c in range(0, cols, step):
                if dem[r, c] != nodata:
                    lon, lat = rasterio.transform.xy(transform, r, c)
                    all_data.append({
                        'location': loc_label,
                        'longitude': lon,
                        'latitude': lat,
                        'elevation': dem[r, c],
                        'slope_deg': slope[r, c],
                        'aspect_deg': aspect[r, c]
                    })

df = pd.DataFrame(all_data)
df.to_csv('combined_mines_features.csv', index=False)
print("Saved sampled features to combined_mines_features.csv")


Processing P5_PAN_Maharashtra_N16_000_E074_000_DEM_30m.tif...
Processing P5_PAN_AndhraPradesh_N18_000_E083_000_DEM_30m.tif...
Processing P5_PAN_Odisha_N21_000_E083_000_DEM_30m.tif...
Processing P5_PAN_Odisha_N21_000_E084_000_DEM_30m.tif...
Processing P5_PAN_Chattisgarh_N22_000_E082_000_DEM_30m.tif...
Processing P5_PAN_Jharkhand_N23_000_E086_000_DEM_30m.tif...
Processing P5_PAN_WestBengal_N23_000_E087_000_DEM_30m.tif...
Processing P5_PAN_MadhyaPradesh_N24_000_E082_000_DEM_30m.tif...
Saved sampled features to combined_mines_features.csv
